2023/3/4<br>
This code is for extracting anatomical parameters and MFA cell by cell

In [1]:
import os
import cv2
import copy
import glob
import common.Watershed_func
import numpy as np
import pandas as pd
import mahotas as mh
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import interpolate, stats
from scipy.signal import savgol_filter
from skimage.measure import regionprops
from sklearn.neighbors import KernelDensity

In [2]:
current_path=os.getcwd()
npz_path=os.path.join(current_path, "npz_file")
target_npz_path=os.path.join(npz_path, "pred_im_concatenation")
segmentation_npz_path=os.path.join(npz_path, "segmentation")
gt_path=os.path.join(current_path, "fig_save", "pred_concatenation")
Polscope_x40=160.256 #nm/pixel

if os.path.exists(segmentation_npz_path)==False:
    os.makedirs(segmentation_npz_path)

#get path of npz files
npz_files=glob.glob(os.path.join(target_npz_path, "*.npz"))

define function

In [3]:
def get_anatomy(nuclei_, lumen_im):
    #extrat cell anatomy
    #calculate region props
    props_cell = regionprops(nuclei_)
    props_lumen = regionprops(lumen_im)

    #prepare dataframe
    cell_label_list=[props_cell[j]["label"] for j in range(len(props_cell))]
    lumen_label_list=[props_lumen[j]["label"] for j in range(len(props_lumen))]

    target_indicators=["area", "perimeter", "eccentricity", "centroid"]
    cell_target_indicators=["cell_area", "cell_perimeter", "cell_eccentricity", "cell_centroid_x", "cell_centroid_y"]
    lumen_target_indicators=[cell_target_indicators[i].replace("cell", "lumen") for i in range(len(cell_target_indicators))]

    df_cell=pd.DataFrame(columns=cell_target_indicators, index=cell_label_list)
    df_lumen=pd.DataFrame(columns=lumen_target_indicators, index=lumen_label_list)
    
    for i, target_indicator in enumerate(target_indicators):
        if target_indicator=="centroid":
            cell_centroid_x_list=[np.int(props_cell[j][target_indicator][1]) for j in range(len(props_cell))]
            cell_centroid_y_list=[np.int(props_cell[j][target_indicator][0]) for j in range(len(props_cell))]
            df_cell.iloc[:, i]=cell_centroid_x_list
            df_cell.iloc[:, i+1]=cell_centroid_y_list

            lumen_centroid_x_list=[np.int(props_lumen[j][target_indicator][1]) for j in range(len(props_lumen))]
            lumen_centroid_y_list=[np.int(props_lumen[j][target_indicator][0]) for j in range(len(props_lumen))]
            df_lumen.iloc[:, i]=lumen_centroid_x_list
            df_lumen.iloc[:, i+1]=lumen_centroid_y_list

        else:
            target_cell_indicator_list=[props_cell[j][target_indicator] for j in range(len(props_cell))]
            df_cell.iloc[:, i]=target_cell_indicator_list

            target_lumen_indicator_list=[props_lumen[j][target_indicator] for j in range(len(props_lumen))]
            df_lumen.iloc[:, i]=target_lumen_indicator_list
        
    #merge dataframe
    df_result=df_cell.join(df_lumen)
    df_result=df_result.replace(np.nan,{'lumen_area':0,'lumen_perimeter':0})

    #add cellwall occupancy and cellwall area on dataframe
    cellwall_occupancy=(df_result["cell_area"]-df_result["lumen_area"])/df_result["cell_area"]
    cellwall_area=df_result["cell_area"]-df_result["lumen_area"]

    df_result["cellwall_area"]=cellwall_area
    df_result["cellwall_occupancy"]=cellwall_occupancy
    
    return df_result

In [4]:
gt_im_files=[]
for curdir, _, files in tqdm(os.walk(gt_path)):
    if len(files)>0:
        for file in files:
            target_path=os.path.join(curdir, file)
            gt_im=cv2.imread(target_path)
            gt_im=gt_im[:, :, 2]
            gt_im=cv2.bitwise_not(gt_im)
            gt_im_files.append(gt_im)

33it [00:06,  5.53it/s]


watershed segmentation and anatomical parameter extraction

In [8]:
df_list=[]
nuclei_list=[]

for gt_im, npz_file in tqdm(zip(gt_im_files, npz_files)):
    #watershed segmentation
    watershed=common.Watershed_func.watershed()
    nuclei, lines=watershed.watershed_segmentation(gt_im)
    nuclei_ = mh.labeled.remove_bordering(nuclei)
    
    #load npz
    npz=np.load(npz_file)
    MFA=npz["MFA"]
    cw_ind=np.where(MFA>0)
    lumen_im=copy.deepcopy(nuclei_)
    lumen_im[cw_ind]=0
    
    #extract anatomy
    df_result=get_anatomy(nuclei_, lumen_im)
    
    #save result
    df_list.append(df_result)
    nuclei_list.append(nuclei_)
    
    sample_name=os.path.splitext(os.path.basename(npz_file))[0]
    np.savez_compressed(os.path.join(segmentation_npz_path, str(sample_name)+".npz"), 
                        nuclei=nuclei_, lumen_im=lumen_im)

0it [00:00, ?it/s]/Users/AR3/anaconda3/envs/YKDL/lib/python3.6/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/Users/AR3/anaconda3/envs/YKDL/lib/python3.6/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See http://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
32it [13:19, 24.37s/it]


cell by cell MFA extraction

In [32]:
df_MFA_list=[]

for i in range(len(npz_files)):
    #load npz
    target_npz=npz_files[i]
    target_df=df_list[i]
    target_nuclei=nuclei_list[i]
    
    npz=np.load(target_npz)
    target_MFA=npz["MFA"]
    
    #set list
    cell_labels=target_df.index.values
    MFA_mean_list=[]
    MFA_mode_list=[]
    MFA_std_list=[]
    
    for cell_label in tqdm(cell_labels):
        #get pixel values of target cell region
        cell_ind=np.where(target_nuclei==cell_label)
        cell_MFA=target_MFA[cell_ind]
        cell_MFA=cell_MFA[cell_MFA>0]
        
        if len(cell_MFA)==0:
            MFA_mean=np.nan
            MFA_std=np.nan
            MFA_mode=np.nan
            
            #save result
            MFA_mean_list.append(MFA_mean)
            MFA_mode_list.append(MFA_mode)
            MFA_std_list.append(MFA_std)
            
        else:
            #kernel density estimation
            X = cell_MFA[:, np.newaxis]
            X_plot = np.linspace(0, 70, 141)[:, np.newaxis]
            kde = KernelDensity(kernel="gaussian", bandwidth=3).fit(X)
            log_dens = kde.score_samples(X_plot)
            dens=np.exp(log_dens)

            #get statistics
            MFA_mean=np.sum(dens*X_plot.flatten())/np.sum(dens)
            MFA_std=np.sqrt(np.sum([dens[j]*((X_plot.flatten()[j]-MFA_mean)**2) for j in range(len(dens))])/np.sum(dens))
            MFA_mode=X_plot[np.argmax(dens)][0]

            #save result
            MFA_mean_list.append(np.round(MFA_mean, 1))
            MFA_mode_list.append(MFA_mode)
            MFA_std_list.append(np.round(MFA_std, 1))
        
    #add columns
    target_df["MFA_mean"]=MFA_mean_list
    target_df["MFA_mode"]=MFA_mode_list
    target_df["MFA_std"]=MFA_std_list
    
    df_MFA_list.append(target_df)

100%|██████████| 3242/3242 [04:31<00:00, 11.94it/s]


In [34]:
#save dataframe
np.savez_compressed(os.path.join(npz_path, "Dataframe_MFA_anatomy.npz"), 
                    df_list=df_MFA_list)

/Users/AR3/anaconda3/envs/YKDL/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
